In [2]:
from multimodalrec.multimodalrec import MultimodalRec

Using TensorFlow backend.


In [3]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import normalize

In [4]:
recmodel = MultimodalRec()

In [5]:
recmodel.organize_multimodal_data(load=True)

User-Movie Latent Factors are extracting...
The sparsity level of training dataset is 95.1%


  3%|▎         | 77/2930 [00:00<00:03, 768.55it/s]

Done.
Visual Representations are extracting...


100%|██████████| 2930/2930 [00:03<00:00, 732.92it/s]

Done.


In [6]:
print(len(recmodel.user_latent))
print(recmodel.user_latent[1].shape)

print(len(recmodel.movie_latent))
print(recmodel.movie_latent[1].shape)

print(len(recmodel.visual_features)) # includes training set
print(recmodel.visual_features[1].shape)

6040
(256,)
2814
(256,)
2930
(30, 2048)


In [7]:
# Create X_train y_train
ratings_df = recmodel.user_item_network.CF_data
pos_ratings_df = ratings_df[ratings_df.Rating>3]
# RefMovs = pos_ratings_df.groupby('User')['Movie'].apply(list)
pos_ratings_df = pos_ratings_df.assign(Likes= lambda x: 1)
# pos_ratings_df['ReferenceMovies'] = pos_ratings_df['User'].map(RefMovs)
print(pos_ratings_df.shape)
pos_ratings_df.head()

(486834, 5)


,User,Movie,Rating,Timestamp,Likes
0,1,1193,5,2000-12-31 22:12:40,1
4,1,2355,5,2001-01-06 23:38:11,1
6,1,2804,5,2000-12-31 22:11:59,1
7,1,594,4,2000-12-31 22:37:48,1
8,1,919,4,2000-12-31 22:22:48,1


In [8]:
# items_as_cols = pos_ratings_df.apply(lambda x: pd.Series(x['ReferenceMovies']),axis=1)
# items_as_cols['orig_index'] = items_as_cols.index
# melted_items = pd.melt(items_as_cols, id_vars='orig_index', var_name='sample_num', value_name='Ref_Movie')
# melted_items.set_index('orig_index', inplace=True)

# pos_ratings_df = pos_ratings_df.merge(melted_items, left_index=True, right_index=True)
# pos_ratings_df.shape

In [9]:
# Create X_train y_train
ratings_df = recmodel.user_item_network.CF_data
neg_ratings_df = ratings_df[ratings_df.Rating<=3]
# RefMovs = neg_ratings_df.groupby('User')['Movie'].apply(list)
neg_ratings_df = neg_ratings_df.assign(Likes= lambda x: 0)
# neg_ratings_df['ReferenceMovies'] = neg_ratings_df['User'].map(RefMovs)
print(neg_ratings_df.shape)
neg_ratings_df.head()

(346981, 5)


,User,Movie,Rating,Timestamp,Likes
1,1,661,3,2000-12-31 22:35:09,0
2,1,914,3,2000-12-31 22:32:48,0
5,1,1197,3,2000-12-31 22:37:48,0
13,1,2687,3,2001-01-06 23:37:48,0
16,1,2321,3,2000-12-31 22:36:45,0


In [10]:
# items_as_cols = neg_ratings_df.apply(lambda x: pd.Series(x['ReferenceMovies']),axis=1)
# items_as_cols['orig_index'] = items_as_cols.index
# melted_items = pd.melt(items_as_cols, id_vars='orig_index', var_name='sample_num', value_name='Ref_Movie')
# melted_items.set_index('orig_index', inplace=True)

# neg_ratings_df = neg_ratings_df.merge(melted_items, left_index=True, right_index=True)
# neg_ratings_df.shape

In [11]:
training_df = pd.concat([pos_ratings_df,neg_ratings_df],axis=0, sort=False)
training_df.shape

(833815, 5)

In [12]:
training_df = training_df.drop(['Timestamp','Rating'], axis=1)
training_df.shape

(833815, 3)

In [13]:
training_df.head(1)

,User,Movie,Likes
0,1,1193,1


In [ ]:
X, y = [], []
np.random.seed(232)
# trainingX = open('trainingX.cvs', 'w')
# trainingy = open('trainingy.cvs', 'w')
# with open('trainingX.cvs', 'a') as f:
for index, row in training_df.iterrows(): 

    normed_user_latent = np.array(recmodel.user_latent[row['User']]) / np.linalg.norm(np.array(recmodel.user_latent[row['User']]))
    #print(normed_user_latent.shape)
    vis_frames = recmodel.visual_features[row['Movie']]
    np.random.shuffle(vis_frames)

    for frame in vis_frames[:5]:
        normed_frame_features = frame / np.linalg.norm(frame)
        #print(normed_frame_features.shape)

        fused_features = np.concatenate([normed_frame_features, normed_user_latent])

#         np.savetxt(f, fused_features, fmt="%d")
#         trainingy.write(str(row['Likes']))

        X.append(fused_features)
        y.append(row['Likes'])
#     break

In [62]:
np.array(X).shape

(5, 2304)

In [63]:
np.array(y).shape

(5,)

In [64]:
X

[array([ 0.00934482,  0.01519698,  0.00724566, ..., -0.02558475,
         0.01077429,  0.03506238]),
 array([ 0.01172481,  0.01144171,  0.0171877 , ..., -0.02558475,
         0.01077429,  0.03506238]),
 array([ 0.01975105,  0.01908408,  0.00967255, ..., -0.02558475,
         0.01077429,  0.03506238]),
 array([ 0.00079263,  0.0038699 ,  0.00473542, ..., -0.02558475,
         0.01077429,  0.03506238]),
 array([ 0.00667729,  0.00568597,  0.01435956, ..., -0.02558475,
         0.01077429,  0.03506238])]

In [46]:
k = recmodel.visual_features[row['Movie']]
k

array([[0.06424791, 0.46066165, 0.07985336, ..., 0.22710806, 1.2884245 ,
        0.20967357],
       [0.35950252, 0.17278245, 0.4138576 , ..., 0.06788354, 0.09639137,
        0.1827299 ],
       [0.5277745 , 0.4246459 , 0.3682531 , ..., 0.04477851, 0.62265044,
        0.13475198],
       ...,
       [0.10783178, 0.12352527, 0.35091925, ..., 0.1705728 , 1.509834  ,
        0.23580384],
       [0.46540892, 0.49070862, 0.31217238, ..., 0.        , 0.55122775,
        0.11096258],
       [0.32453746, 0.280178  , 0.27946118, ..., 0.00317025, 0.7611169 ,
        0.27896708]], dtype=float32)

In [49]:
np.random.shuffle(k)
k[:5]

array([[0.05805467, 0.18774937, 0.11471506, ..., 0.13911499, 0.43751204,
        0.48608038],
       [0.35528803, 0.34329048, 0.17399295, ..., 0.        , 0.5718152 ,
        0.4152525 ],
       [0.01682228, 0.08213215, 0.1005014 , ..., 0.23561662, 0.2803183 ,
        0.36174986],
       [0.5277745 , 0.4246459 , 0.3682531 , ..., 0.04477851, 0.62265044,
        0.13475198],
       [0.10783178, 0.12352527, 0.35091925, ..., 0.1705728 , 1.509834  ,
        0.23580384]], dtype=float32)